In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_sub = pd.read_csv('sample_submission.csv')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
label = LabelEncoder()

In [6]:
df_train_temp = df_train.copy()

In [7]:
df_train_temp['Target'] = label.fit_transform(df_train_temp['Target'])

In [8]:
df_train_temp['Target'].value_counts()

Target
2    36282
0    25296
1    14940
Name: count, dtype: int64

In [9]:
for i, j in zip(df_train_temp.corr()['Target'], df_train_temp.columns):
    if -0.09<i<0.09:
        df_train = df_train.drop(j, axis=1)
        df_test = df_test.drop(j, axis=1)

In [10]:
X = df_train.drop('Target', axis=1)
y= df_train['Target']

In [12]:
y

0        Graduate
1         Dropout
2         Dropout
3        Enrolled
4        Graduate
           ...   
76513    Graduate
76514    Graduate
76515    Enrolled
76516     Dropout
76517    Graduate
Name: Target, Length: 76518, dtype: object

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scaler = StandardScaler()

In [15]:
X = scaler.fit_transform(X)

In [16]:
df_test = scaler.transform(df_test)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
model = RandomForestClassifier()

In [19]:
hyper_param = {
    'n_estimators': [65, 75, 90],
    'criterion': ['gini', 'entropy'],
    'max_depth': [4, 6 , 8],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2],
    'min_weight_fraction_leaf': [0.0, 0.1]
}

In [20]:
full_model = GridSearchCV(model , hyper_param, cv=3 , verbose=2)

In [21]:
full_model.fit(X, y)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=65; total time=   1.8s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=65; total time=   1.6s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=65; total time=   1.6s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=75; total time=   1.9s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=75; total time=   1.9s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=75; total time=   1.8s
[CV] END criterion=gini, max_depth=4, min_samples_leaf=1, min_sampl

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 6, 8], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3, 4],
                         'min_weight_fraction_leaf': [0.0, 0.1],
                         'n_estimators': [65, 75, 90]},
             verbose=2)

In [22]:
y_pred = full_model.predict(df_test)

In [24]:
y_pred

array(['Dropout', 'Graduate', 'Graduate', ..., 'Dropout', 'Dropout',
       'Dropout'], dtype=object)

In [26]:
df_sub['Target'] = y_pred

In [27]:
df_sub.to_csv('Hayko_kyanqs_mi_ker.csv' , index = False, index_label = False)